In [1]:
Cluster=False

To access s3 files, the first step is setting AWS credential.

In [2]:
if Cluster:
    %run Credentials.ipynb

    # stop sc before restarting it.
    #sc.stop()

    from pyspark import SparkContext,SparkConf
    sparkConfig=SparkConf()
    sparkConfig.set("spark.executor.memory","20g")
    sparkConfig.set("spark.worker.memory","20g")
    sparkConfig.set("spark.driver.cores","8")
    sparkConfig.set("spark.python.worker.memory","20g")
    sparkConfig.getAll()

    sc=SparkContext(conf=sparkConfig)

In [3]:
# test sc
RDD=sc.parallelize(range(100))
RDD.count()

100

In [4]:
if Cluster:
    s3helper.set_credential(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)
    # Then open the bucket that has your files.
    s3helper.open_bucket('yoav-faces')

    # Now you can list your files in the bucket.

    print s3helper.ls()
    tarnames=s3helper.ls('output/')
    print 'tarnames=',tarnames[:10]
    
    !mkdir /mnt/output
    %cd /mnt/output
    !ls -lrt data1/output
    !rm data1/output/*.pkl

    workdir='/mnt/output/data1/output/'
    %cd '/mnt/output'

else:  #run on local
    tarnames=['Bernie.tgz']
    print 'tarnames=',tarnames

    workdir='/Users/yoavfreund/projects/DimensionalityEstimation/src/Spark-KMeans++/data1/output/'
    %cd '/Users/yoavfreund/projects/DimensionalityEstimation/src/Spark-KMeans++'
    !cp /Users/yoavfreund/projects/DimensionalityEstimation/output/Bernie.tgz .
    !tar -xzvf Bernie.tgz
    !ls $workdir

tarnames= ['Bernie.tgz']
/Users/yoavfreund/projects/DimensionalityEstimation/src/Spark-KMeans++
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows0.pkl
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows1.pkl
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows2.pkl
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows3.pkl
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows4.pkl
x data1/output/Bernie Sanders Promposals-Ks2v3LSd29U_windows5.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows0.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows1.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows2.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows3.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows4.pkl
Bernie Sanders Promposals-Ks2v3LSd29U_windows5.pkl


In [5]:
import numpy as np
"""Code for packing and unpacking a numpy array into a byte array.
   the array is flattened if it is not 1D.
   This is intended to be used as the interface for storing 
   
   This code is intended to be used to store numpy array as fields in a dataframe and then store the 
   dataframes in a parquet file.
"""

def packArray(a):
    if type(a)!=np.ndarray:
        raise Exception("input to packArray should be numpy.ndarray. It is instead "+str(type(a)))
    return bytearray(a.tobytes())
def unpackArray(x,data_type=np.int16):
    return np.frombuffer(x,dtype=data_type)

In [8]:
from sys import getsizeof
from pyspark.sql import Row, SQLContext
import sys,traceback
from glob import glob
import pickle
import re
import numpy as np
from time import time
from pyspark import StorageLevel
sqlC=SQLContext(sc)

ALL=[]
cumul=0

for tar_index in range(len(tarnames)):
    tar_path=tarnames[tar_index]
    if Cluster:
        tarname=tar_path[7:]
        print 'starting on ',tar_index,tarname
    else:
        tarname=tar_path
        print 'Running in local mode',tar_index,tarname
    !date
    try:
        if Cluster:
            s3helper.get_file(tar_path)

            tarname=tar_path
            !ls -l "$tarname"
            !tar -xzvf "$tarname"
            !rm "$tarname"
            !mv *.pkl $workdir
        
        video_names={}
        video_index=0

        pattern=re.compile(r'.*/([^/]+)_windows(\d+)\.pkl')

        List=glob(workdir+'*')
        data=[]
        ts=[]

        for file in List:
            ts.append(time())
            match=re.search(pattern,file)
            if match:
                video_name=match.group(1)
                if not video_name in video_names.keys():        
                    video_names[video_name]=video_index
                    video_index+=1
                video_num=video_names[video_name]
                window_num=int(match.group(2))
            else:
                print 'COULD NOT FIND NUMBER IN',file
                continue

            In = pickle.load(open(file,'r'))
            print window_num,len(In),
            Full=[]
            for f in In:
                Lim=packArray(np.array(f[-1],dtype=np.int16))
                row=Row(video_num=int(video_num), track_num=int(window_num),frame_num=int(f[0]),
                        ulx=int(f[1]),uly=int(f[2]),size=int(f[3]),image=Lim)
                Full.append(row)
                
            In=[]
            data=data+Full
            Full=[]
            print window_num,len(List),len(data)
            #break
            
        print 'size of data=',getsizeof(data)

        ts.append(time())
        rdd=sc.parallelize(data,10)
        ts.append(time())
        New=sqlC.createDataFrame(rdd).persist(StorageLevel.MEMORY_AND_DISK_SER)
        ts.append(time())
        _size=New.count()
        ts.append(time())
        ALL.append(New)
        
        print [ts[i+1]-ts[i] for i in range(len(ts)-1)]
        cumul += _size
        print 'finished ',tarname,
        print tar_index,'added',_size,'for a total of',cumul
        print '='*70
        break

    except:
        print "Exception while reading file:",file
        print '-'*60
        traceback.print_exc(file=sys.stdout)
        print '-'*60

    !rm /mnt/output/data1/output/*.pkl
    !rm *.pkl

 Running in local mode 0 Bernie.tgz
Tue Aug  9 08:58:38 PDT 2016
0 994 0 6 994
1 846 1 6 1840
2 228 2 6 2068
3 101 3 6 2169
4 254 4 6 2423
5 106 5 6 2529
size of data= 20304
[1.1028499603271484, 0.9427151679992676, 0.2554168701171875, 0.1090090274810791, 0.2841639518737793, 0.11392402648925781, 3.166512966156006, 1.3507020473480225, 9.4113130569458]
finished  Bernie.tgz 0 added 2529 for a total of 2529


In [9]:
#New=sqlC.createDataFrame(rdd)
print type(New)
print New.count()
LX=New.take(5)
for X in LX:
    print X.frame_num

<class 'pyspark.sql.dataframe.DataFrame'>
2529
3
4
5
6
7


In [10]:
parquet_file="TestFaces.parquet"
!rm -rf $parquet_file
New.write.save(parquet_file)

!ls -l $parquet_file

total 240488
-rw-r--r--  1 yoavfreund  staff         0 Aug  9 08:25 _SUCCESS
-rw-r--r--  1 yoavfreund  staff       668 Aug  9 08:25 _common_metadata
-rw-r--r--  1 yoavfreund  staff   3609419 Aug  9 08:25 _metadata
-rw-r--r--  1 yoavfreund  staff  10241699 Aug  9 08:25 part-r-00000-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  10529844 Aug  9 08:25 part-r-00001-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  11369446 Aug  9 08:25 part-r-00002-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  10930243 Aug  9 08:25 part-r-00003-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  10834174 Aug  9 08:25 part-r-00004-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  10959497 Aug  9 08:25 part-r-00005-15f372be-80f1-4ad0-9e4c-1dfcc7a6f1c4.gz.parquet
-rw-r--r--  1 yoavfreund  staff  10870156 Aug  9 08:25 part-r-00006-15f372be-80f1-4ad0-9e4

In [14]:
df = sqlContext.sql("SELECT ulx,uly FROM parquet.`%s`"%parquet_file)
df.head(5)

[Row(ulx=567, uly=228),
 Row(ulx=566, uly=227),
 Row(ulx=565, uly=225),
 Row(ulx=564, uly=223),
 Row(ulx=565, uly=222)]

## Parquet Files

In [ ]:
s3helper.open_bucket("mas-dse-public")

files = s3helper.load_path('/Weather/US_Weather.parquet', '/US_Weather.parquet')
files[:10]

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext(master=master_url)
sqlContext = SQLContext(sc)

In [ ]:
df = sqlContext.sql("SELECT station, measurement FROM parquet.`/US_Weather.parquet`")
df.head()